### 1. Prepare Account to scrape Data from Study4


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

#### 1.1 Login

In [ ]:
# Sử dụng Selenium để đăng nhập
options = webdriver.ChromeOptions()
options.add_argument("--headless")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://study4.com/login/")

#Đăng nhập bằng facebook account
driver.find_element(By.CLASS_NAME,"f-login-button").click()

email_input = driver.find_element(By.ID,"email")
email_input.send_keys("nmjuyh26@gmail.com")

password_input = driver.find_element(By.ID,"pass")
password_input.send_keys("tientumega")

driver.find_element(By.ID,"loginbutton").click()
time.sleep(3)

driver.find_element(By.XPATH,"//div[@role='button']").click()
time.sleep(5)

### 1.2 Do all tests for signed Account

In [ ]:
with open('tests.json') as file:
    data = json.load(file)
for row in data:
    url = row['test_url']
    driver.get(url)
    driver.find_element(By.ID,'').click()

### Stop Selenium

In [ ]:
driver.quit()